In [51]:
import requests
from bs4 import BeautifulSoup
from geopy import geocoders  
from geopy.geocoders import Nominatim
from datetime import datetime
from geopy.geocoders import Nominatim
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

def get_city_coordinates(city_name):
    """
    Returns the latitude and longitude of a given French city separately.

    :param city_name: Name of the city (string)
    :return: (latitude, longitude) as separate values or (None, None) if not found
    """

    app = Nominatim(user_agent="tutorial")
    location = app.geocode(city_name + ", France")  # Ensure the search is limited to France
    
    if location:
        return location.latitude, location.longitude
    else:
        return None, None  # City not found

In [52]:
range=60000
# cities=['Aix-en-Provence','Marseille','Avignon','Paris','Digne-les-Bains','Lyon']
cities=['Aix-en-Provence']
get_city_coordinates(cities[0])

(43.5298424, 5.4474738)

In [53]:
# Path to your Chrome WebDriver
chrome_driver_path = "/path/to/chromedriver"

# Configure Selenium to run in headless mode
options = webdriver.ChromeOptions()
options.add_argument("--headless")

# Start the WebDriver
service = Service(chrome_driver_path)
#driver = webdriver.Chrome(service=service, options=options)

from chromedriver_py import binary_path # this will get you the path variable

svc = webdriver.ChromeService(executable_path=binary_path)
driver = webdriver.Chrome(service=svc, options=options)
rows=[]
stages_total=0
for l in cities:
    latitude, longitude = get_city_coordinates(l)
    if latitude is None:
        break
    # Open the URL
    encore=True
    stages = 0
    page=1
    print(f"On cherche des stages dans unm rayon de {range/1000} kms de {l}")
    while(encore):
        print('#')

        url = f"https://1eleve1stage.education.gouv.fr/offres-de-stage?latitude={latitude}&longitude={longitude}&city={l}&radius={range}&page={page}"
        driver.get(url)

        # Wait for the JavaScript-generated content to load (adjust timeout as needed)
        WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "row-link")))


        # Get the fully rendered page source
        html = driver.page_source

        # Parse with BeautifulSoup
        soup = BeautifulSoup(html, "html.parser")

        cards = soup.find_all("div",{"class":'fr-card--full-text'})
        for card in cards:
            #soupcard = BeautifulSoup(card, 'html.parser')

            soupcard = BeautifulSoup(str(card), 'html.parser')
            id = int(soupcard.find('div', class_='fr-card')['data-test-id'])
            sector = soupcard.find('div', class_='sector').text.strip()
            desc= soupcard.find('div', class_='fr-mb-2w').text.strip()
            org= soupcard.find('div', class_='fr-mb-1w').text.strip()
            city = soupcard.find('div', class_='test-city').text.strip()
            #print (f"{data_test_id} - {sector} - {city}")    
            rows.append({"ID": id, "sector": sector, "org": org, "desc": desc, "ville": city,'lien':f'https://1eleve1stage.education.gouv.fr/offres-de-stage/{id}'})  # Store data as dict
            stages+=1
        
        encore= len(soup.find_all("a",{"class":'fr-pagination__link--next'}))>0
        if(encore):
            print(f'PAGE {page} DONE - NEXT')
            page+=1
        else:
            print(f'DONE for LOCATION {l} : {stages} recuperes')
    stages_total+=stages
print(f'DONE for ALL {len(cities)} LOCATIONS : {stages_total} recuperes')
df = pd.DataFrame(rows) 
# Close the browser session
driver.quit()

df.drop_duplicates(inplace=True)
interests = ['Arts du spectacle','Audiovisuel']
display(df[df.sector.isin(interests)])


On cherche des stages dans unm rayon de 60.0 kms de Aix-en-Provence
#
PAGE 1 DONE - NEXT
#
PAGE 2 DONE - NEXT
#
PAGE 3 DONE - NEXT
#
PAGE 4 DONE - NEXT
#
PAGE 5 DONE - NEXT
#
PAGE 6 DONE - NEXT
#
PAGE 7 DONE - NEXT
#
PAGE 8 DONE - NEXT
#
PAGE 9 DONE - NEXT
#
DONE for LOCATION Aix-en-Provence : 286 recuperes
DONE for ALL 1 LOCATIONS : 286 recuperes


,ID,sector,org,desc,ville,lien
57,36544,Audiovisuel,Audiovisuel,Théories et pratiques du cinéma,Les pennes-mirabeau,https://1eleve1stage.education.gouv.fr/offres-...
147,36339,Arts du spectacle,Arts du spectacle,Mise en oeuvre d'un projet artistique,Martigues,https://1eleve1stage.education.gouv.fr/offres-...
153,35591,Arts du spectacle,Arts du spectacle,Assistant mise en scène de spectacles - A...,Cassis,https://1eleve1stage.education.gouv.fr/offres-...
166,35592,Arts du spectacle,Arts du spectacle,Assistant mise en scène de spectacles - A...,La ciotat,https://1eleve1stage.education.gouv.fr/offres-...


In [46]:
df.to_pickle(f'stages-{datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.pkl')

In [47]:
df.sector.value_counts()

sector
Enseignement                                        601
Bâtiment et travaux publics (BTP)                   131
Informatique et réseaux                              82
Paramédical                                          65
Fonction publique                                    54
Commerce et distribution                             53
Verre, béton, céramique                              49
Architecture, urbanisme et paysage                   41
Électronique                                         33
Social                                               27
Comptabilité, gestion, ressources humaines           25
Agriculture                                          22
Hôtellerie, restauration                             21
Santé                                                17
Droit et justice                                     16
Énergie                                              14
Automobile                                           13
Sport                                    